### Day 16: Reindeer Maze

Link: https://adventofcode.com/2024/day/16

We can solve this problem by using Dijkstra's Algorithm, where we gradually make progress through the grid, prioritizing lower accumulated scores first. We can also add a set of visited states to avoid revisiting the same place with the same direction multiple times, as that would only increase the score we want to minimize. To make it easier, I also created a heap queue class supporting a custom sorting method that is more intuitive than Python's built-in `heapq` methods. This approach should correspond to `O(n log n)` time.

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
import heapq
import itertools
import typing as t


T = t.TypeVar("T")


class HeapQueue(t.Generic[T]):
    def __init__(self, key: t.Optional[t.Callable[[T], float]] = None) -> None:
        self._heap: list[tuple[float, int, T]] = []
        self._key = key or (lambda x: x)
        self._counter = itertools.count()  # Unique counter to ensure stable ordering

    def add(self, item: T) -> None:
        count = next(self._counter)
        heapq.heappush(self._heap, (self._key(item), count, item))

    def remove(self) -> T:
        if self.is_empty():
            raise IndexError("Empty heap")

        return heapq.heappop(self._heap)[-1]

    def peek(self) -> T:
        if self.is_empty():
            raise IndexError("Empty heap")

        return self._heap[0][-1]

    def is_empty(self) -> bool:
        return len(self._heap) == 0

    def __len__(self) -> int:
        return len(self._heap)

In [ ]:
from dataclasses import dataclass


grid: list[list[str]] = []


for line in lines:
    grid_row = list(line.strip())
    grid.append(grid_row)


def find_start_position() -> tuple[int, int]:
    for row in range(len(grid)):
        for column, cell in enumerate(grid[row]):
            if cell == "S":
                return row, column

    raise Exception("Start position not found")


@dataclass
class Reindeer:
    row: int
    column: int
    direction: int
    score: int


row, column = find_start_position()
actions = [
    (0, 1), # Right
    (1, 0), # Down
    (0, -1), # Left
    (-1, 0), # Up
]
queue: HeapQueue[Reindeer] = HeapQueue(key=lambda reindeer: reindeer.score)
queue.add(Reindeer(row=row, column=column, direction=0, score=0))


def turn_right(direction: int) -> int:
    return (direction + 1) % len(actions)


def turn_left(direction: int) -> int:
    return (direction - 1) % len(actions)


visited: set[tuple[int, int, int]] = set()
min_score = float("inf")


while not queue.is_empty():
    reindeer = queue.remove()

    if (reindeer.row, reindeer.column, reindeer.direction) in visited:
        continue

    visited.add((reindeer.row, reindeer.column, reindeer.direction))

    if grid[reindeer.row][reindeer.column] == "E":
        min_score = reindeer.score
        break

    new_row, new_column = reindeer.row + actions[reindeer.direction][0], reindeer.column + actions[reindeer.direction][1]

    if grid[new_row][new_column] != "#":
        queue.add(Reindeer(
            row=new_row, column=new_column, direction=reindeer.direction, score=reindeer.score + 1
        ))

    queue.add(Reindeer(
        row=reindeer.row,
        column=reindeer.column,
        direction=turn_right(reindeer.direction),
        score=reindeer.score + 1_000,
    ))
    queue.add(Reindeer(
        row=reindeer.row,
        column=reindeer.column,
        direction=turn_left(reindeer.direction),
        score=reindeer.score + 1_000,
    ))


print(min_score)